In [4]:
from torch.utils.data import DataLoader

import mapd
import torchvision
from torchvision import transforms

from mapd.probes.make_probe_suites import make_probe_suites
from mapd.probes.utils.idx_dataset import IDXDataset
from mapd.proxies.proxy_calculator import ProxyCalculator
from mapd.utils.make_dataloaders import make_dataloaders

In [5]:
MNIST_ROOT = "data"
torchvision.datasets.MNIST(root=MNIST_ROOT, download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train

In [6]:
from torchvision.datasets import MNIST

In [7]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x


model = Net()

In [8]:
from typing import Any
import lightning as L
from torch.nn import functional as F
from torch.optim import SGD
import torch


class ResNet18(mapd.MAPDModule):
    def __init__(
            self,
            max_epochs: int = 10,
            lr: float = 0.05,
            momentum: float = 0.9,
            weight_decay: float = 0.0005
    ):
        super().__init__()
        self.model = model

        self.max_epochs = max_epochs
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay

        self.save_hyperparameters(ignore=["model"])

    def mapd_settings(self):
        return {
            "proxy_metric": "loss",
            "proxy_metric_direction": "minimize",
        }

    def forward(self, x):
        return self.model(x)

    def batch_loss(self, logits, y) -> torch.Tensor:
        return F.cross_entropy(logits, y, reduction="none")

    def training_step(self, batch, batch_idx):
        x, y = batch

        logits = self.forward(x)
        loss = self.batch_loss(logits, y).mean()
        self.mapd_log(logits, y)

        return loss

    def validation_step(self, batch, batch_idx, dataloader_idx: int = 0):
        x, y = batch

        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        if dataloader_idx == 0:
            self.mapd_log(logits, y)

        return loss

    def configure_optimizers(self):
        optimizer = SGD(
            self.parameters(),
            lr=self.lr
        )

        return {"optimizer": optimizer}

In [10]:
from torch.utils.data import random_split

module = ResNet18()

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
mnist_test = MNIST(MNIST_ROOT, train=False, transform=transform)
mnist_predict = MNIST(MNIST_ROOT, train=False, transform=transform)
mnist_full = MNIST(MNIST_ROOT, train=True, transform=transform)
mnist_train, mnist_val = random_split(mnist_full, [55000, 5000])
mnist_train = IDXDataset(mnist_train)

dl = DataLoader(mnist_train, batch_size=512, shuffle=True, num_workers=16, prefetch_factor=8)

torch.set_float32_matmul_precision('medium')

trainer_proxy = L.Trainer(accelerator="gpu", max_epochs=100)
trainer_probes = L.Trainer(accelerator="gpu", max_epochs=100)

# Proxy
trainer_proxy.fit(module.as_proxies(), train_dataloaders=dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jonashoffmann/.applications/miniconda3/envs/mapd-pkg/lib/python3.9/site-packages/lightning/pytorch/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | Net  | 21.8 K
-------------------------------
21.8 K    Trainable params
0         Non-trainable params
21.8 K    Total params
0.087     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

TypeError: conv2d() received an invalid combination of arguments - got (list, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list of [Tensor, Tensor]!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)


In [ ]:
pc = ProxyCalculator("proxies", "proxy_metric")

probe_suite_ds = make_probe_suites(mnist_train, 10, pc, num_probes=500)

In [ ]:
dl_probes = DataLoader(mnist_train, batch_size=512, shuffle=True, num_workers=16, prefetch_factor=8)

val_dataloader = DataLoader(IDXDataset(mnist_val), batch_size=512, shuffle=True, num_workers=16, prefetch_factor=8)
val_dataloaders = make_dataloaders([val_dataloader], probe_suite_ds)

trainer_probes.fit(module.as_probes(probe_suite_ds), train_dataloaders=dl_probes, val_dataloaders=val_dataloaders)